## Assignment 1 
#### CS395: Deep learning
#### 2/26/18
#### By Alex Ram

### The code bellow is used for all three parts

In [3]:
#imports
from keras.applications.resnet50 import ResNet50 
from keras.preprocessing.image import img_to_array
from IPython.display import Image
import cv2
import random
import numpy as np
from keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

In [4]:
#code bellow was inspired by this tutorial: https://www.pyimagesearch.com/2017/12/11/image-classification-with-keras-and-deep-learning/
#This will take the images and covert it to an array, also the labels
#path to the pictures
PATH = "../../png/"
trainData = []
trainLabels = []

testData = []
testLabels = []
 
# grab the image paths and randomly shuffle them
imagePaths = open(PATH + "filelist.txt").readlines()

def convertImageToArray(imagePath):
    image = cv2.imread(PATH + imagePath.strip())
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image


for i in range(len(imagePaths)):
    if i % 80 < 60:
        #image coverted to array and appended to data set
        trainData.append(convertImageToArray(imagePaths[i]))
        
        #labels of images appended to labels dataset
        trainLabels.append(imagePaths[i].split("/"))
        trainLabels[-1][1] = trainLabels[-1][1].strip()
    else:
        testData.append(convertImageToArray(imagePaths[i]))
        
        testLabels.append(imagePaths[i].split("/"))
        testLabels[-1][1] = testLabels[-1][1].strip()
        
#covert label lists to numpy arrays to use later
trainLabels = np.array(trainLabels)
testLabels = np.array(testLabels)
#get a set of just each type of lable
uniqueLabels = set(trainLabels[:,0])

## Part 1:
### Using a pretrained network on imageNet to classify the sketches data set by training a classifier 

In [3]:
#model found here: https://keras.io/applications/#resnet50
model = ResNet50(include_top=False, weights='imagenet')

In [4]:
trainFeatures = []
testFeatures = []

#feature extraction from the image arrays using the resnet50 model
for i in range(len(trainData)):
    trainFeatures.append(model.predict(trainData[i]))
    
for i in range(len(testData)):
    testFeatures.append(model.predict(testData[i]))

#squeeze 'em
trainFeatures = np.squeeze(trainFeatures)
testFeatures = np.squeeze(testFeatures)

In [5]:
#code bellow inspired by https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

#Creates the classifier
def createClassifier():
    model = Sequential()
    model.add(Dense(250, input_dim=2048, activation='softmax'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# encode class values as integers
def convertToDummy(labels):
    encoder = LabelEncoder()
    encoder.fit(labels[:,0])
    encoded_Y = encoder.transform(labels[:,0])
    # convert integers to dummy variables (i.e. one hot encoded)
    dummy_y = np_utils.to_categorical(encoded_Y)
    return dummy_y

trainDummyLabels = convertToDummy(trainLabels)
testDummyLabels = convertToDummy(testLabels)

estimator = createClassifier()
estimator.fit(trainFeatures, trainDummyLabels, validation_data=(testFeatures, testDummyLabels), epochs=50, batch_size=10)


Train on 15000 samples, validate on 5000 samples
Epoch 1/50
15000/15000 [==============================] - 7s 450us/step - loss: 3.3138 - acc: 0.3342 - val_loss: 2.7167 - val_acc: 0.4456
Epoch 2/50
15000/15000 [==============================] - 6s 423us/step - loss: 2.3616 - acc: 0.5188 - val_loss: 2.7004 - val_acc: 0.4782
Epoch 3/50
15000/15000 [==============================] - 6s 415us/step - loss: 2.1370 - acc: 0.5711 - val_loss: 2.7349 - val_acc: 0.5066
Epoch 4/50
15000/15000 [==============================] - 6s 422us/step - loss: 2.0377 - acc: 0.6013 - val_loss: 2.7582 - val_acc: 0.5192
Epoch 5/50
15000/15000 [==============================] - 6s 404us/step - loss: 1.9444 - acc: 0.6286 - val_loss: 2.7976 - val_acc: 0.5404
Epoch 6/50
15000/15000 [==============================] - 7s 449us/step - loss: 1.8715 - acc: 0.6456 - val_loss: 2.8239 - val_acc: 0.5354
Epoch 7/50
15000/15000 [==============================] - 6s 422us/step - loss: 1.8245 - acc: 0.6581 - val_loss: 2.8867 - v

## Results

### For all parts bellow I found out after running the code that I could have used the call back method in keras to show the best validation accuracy after all epochs. But for this run it looks like ~58%. Compared to 70-80% being te best performace, that's pretty good.

## Part 2:
### Allowing the entire pretrained network to be trained on the sketches data set

In [ ]:
#I'm taking the resnet50 model defined in the previous part and adjustiong to have the classifier added to it.
#This will allow the entire network to be trained in one go
#code inspired by https://keras.io/applications/#resnet50

x = model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(250, activation='softmax')(x)

# this is the model we will train
modelWithClassifier = Model(inputs=model.input, outputs=predictions)

In [ ]:
modelWithClassifier.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#squeeze/ convert to numpy array so the network can take in the data
trainData = np.squeeze(np.array(trainData))
testData = np.squeeze(np.array(testData))

#train the netwrok
modelWithClassifier.fit(trainData, trainDummyLabels, validation_data=(testData, testDummyLabels), epochs=50, batch_size=50)

Train on 15000 samples, validate on 5000 samples
Epoch 1/50
15000/15000 [==============================] - 463s 31ms/step - loss: 4.6903 - acc: 0.0667 - val_loss: 9.9801 - val_acc: 0.0098
Epoch 2/50
15000/15000 [==============================] - 461s 31ms/step - loss: 2.9415 - acc: 0.2991 - val_loss: 4.3576 - val_acc: 0.1468
Epoch 3/50
15000/15000 [==============================] - 462s 31ms/step - loss: 2.0532 - acc: 0.4795 - val_loss: 2.7733 - val_acc: 0.3808
Epoch 4/50
15000/15000 [==============================] - 462s 31ms/step - loss: 1.5415 - acc: 0.5877 - val_loss: 2.4009 - val_acc: 0.4666
Epoch 5/50
15000/15000 [==============================] - 463s 31ms/step - loss: 1.1677 - acc: 0.6771 - val_loss: 1.8604 - val_acc: 0.5408
Epoch 6/50
15000/15000 [==============================] - 462s 31ms/step - loss: 0.8591 - acc: 0.7500 - val_loss: 2.6971 - val_acc: 0.4016
Epoch 7/50
15000/15000 [==============================] - 463s 31ms/step - loss: 0.6082 - acc: 0.8147 - val_loss: 2.6

## Results:
### For part two, the kernel interupted right before epoch 50 ended, it looks like the best validation accuracy is ~60% which is better than on part one. This is what we expected.

## Part 3:
### Initializing the network with _____ and training from scratch on the sketches data set

In [ ]:
def convertToDummy(labels):
    encoder = LabelEncoder()
    encoder.fit(labels[:,0])
    encoded_Y = encoder.transform(labels[:,0])
    # convert integers to dummy variables (i.e. one hot encoded)
    dummy_y = np_utils.to_categorical(encoded_Y)
    return dummy_y

trainDummyLabels = convertToDummy(trainLabels)
testDummyLabels = convertToDummy(testLabels)

trainData = np.squeeze(np.array(trainData))
testData = np.squeeze(np.array(testData))

In [ ]:
modelFromScratch = ResNet50(include_top=True, weights=None, classes=250)
modelFromScratch.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
modelFromScratch.fit(trainData, trainDummyLabels, validation_data=(testData, testDummyLabels), epochs=50, batch_size=50)

Train on 15000 samples, validate on 5000 samples
Epoch 1/50
15000/15000 [==============================] - 468s 31ms/step - loss: 5.6809 - acc: 0.0169 - val_loss: 15.9852 - val_acc: 0.0040
Epoch 2/50
15000/15000 [==============================] - 461s 31ms/step - loss: 4.8691 - acc: 0.0555 - val_loss: 11.5775 - val_acc: 0.0134
Epoch 3/50
15000/15000 [==============================] - 461s 31ms/step - loss: 4.2866 - acc: 0.1264 - val_loss: 16.0536 - val_acc: 0.0040
Epoch 4/50
15000/15000 [==============================] - 461s 31ms/step - loss: 3.8910 - acc: 0.1883 - val_loss: 6.8751 - val_acc: 0.0940
Epoch 5/50
15000/15000 [==============================] - 462s 31ms/step - loss: 3.6472 - acc: 0.2365 - val_loss: 15.5088 - val_acc: 0.0014
Epoch 6/50
15000/15000 [==============================] - 462s 31ms/step - loss: 3.2558 - acc: 0.3096 - val_loss: 9.6328 - val_acc: 0.0138
Epoch 7/50
15000/15000 [==============================] - 462s 31ms/step - loss: 2.9382 - acc: 0.3584 - val_loss:

## Results:
### For part 3 I terminated the kernel because I am sharing the Azure GPU with my group members for the final project and someone else wanted to use it. I figured 37 epochs was enough for this assignment. 2 or 3 times as many would be what I wanted for a well trained network, but was 4-8 hours away. You can see rapid changes in validation accuracy, which is due to the algorithm exploring the space. The best accuracy seen was ~50%, but if continued for longer you might see the network reach >60%, which was what the part 2 network produced.